# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
import requests
import sys
import config
import time
import json
import mysql.connector
import pandas as pd
import matplotlib as plt
from mysql.connector import errorcode
%load_ext autoreload

%autoreload

In [2]:

# Your code here; use a function or loop to retrieve all the results from your original request

def yelp_review_call(api_key, biz_id):
    url = f'https://api.yelp.com/v3/businesses/{biz_id}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers)
    return response

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    return response


def get_businesses(url_params, api_key):
    response = yelp_call(url_params, api_key)
    print(response.json().keys())
    if 'businesses' in response.json():
        data = response.json()['businesses']
    else:
        raise KeyError
    return data

def all_results(url_params, api_key):
    #declare url here
    #NOTE refactor into Config later
    response = yelp_call(url_params, api_key)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(get_businesses(url_params, api_key))
        time.sleep(.5) #Wait a second
        cur += 50
    return results

term = 'bbq'
location = 'Nashville TN'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, config.api_key)


print(len(df))

1200 total matches found.
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
dict_keys(['businesses', 'total', 'region'])
20


In [3]:
biz_id = df[0][0]["id"]
response = yelp_review_call(config.api_key, biz_id)

In [4]:
#method for connecting to yelp database, return tuple of cnx/c to avoid scoping
def connect_to_yelp():
    cnx = mysql.connector.connect(**config.config)
    c = cnx.cursor()
    statement = """USE yelp"""
    c.execute(statement)
    return (c, cnx)
    

In [5]:
test_id = df[0][0]["id"]
response = yelp_review_call(config.api_key, test_id)

In [6]:
reviews = json.loads(response.text)
reviews['reviews'][0]
#time created, id, rating, {foregin key}

{'id': '9ZwejxZMZJnccTZQDWoaOg',
 'url': 'https://www.yelp.com/biz/peg-leg-porker-nashville?adjust_creative=OGWT2CuDRacTCmIKWxKLCg&hrid=9ZwejxZMZJnccTZQDWoaOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=OGWT2CuDRacTCmIKWxKLCg',
 'text': "Wow! Simply delicious! Dry rub ribs fall off the bone. Pork sandwich was the best. And Smoked green beans we're new to me and absolutely amazing.\n\nFor my...",
 'rating': 5,
 'time_created': '2019-07-23 18:39:18',
 'user': {'id': 'wO2MJ75uYkzwRm5QZ1cr6w',
  'profile_url': 'https://www.yelp.com/user_details?userid=wO2MJ75uYkzwRm5QZ1cr6w',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/KZXlVoUHcyiZbSnxTsWcRA/o.jpg',
  'name': 'Matt K.'}}

In [7]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(**config.config)
c = cnx.cursor()

c.execute("""DROP DATABASE IF EXISTS yelp""")

DB_NAME = 'yelp'

def create_database(cursor):
    """Creates Database and Catch errors"""
    try:
        c.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    c.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    #if DB doesn't exist, create the databse for you
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(c)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

        

TABLES = {}
TABLES['restaurants'] = """
        CREATE TABLE `restaurants` (
           `restaurant_id` varchar(32) NOT NULL,
           `name` TEXT NOT NULL,
           `rating` DECIMAL(2,1),
           `price` TEXT,
           PRIMARY KEY(restaurant_id)
        ) ENGINE=InnoDB
        """

TABLES['reviews'] = """
        CREATE TABLE `reviews` (
           `review_id` VARCHAR(32) NOT NULL,
           `time_created` DATE NOT NULL,
           `review_text` TINYTEXT,
           `user_rating` DECIMAL(2,1),
           `restaurant_id` VARCHAR(32),
           PRIMARY KEY(review_id),
           FOREIGN KEY(restaurant_id) REFERENCES restaurants(restaurant_id)
        ) ENGINE=InnoDB
        """

for table_name in TABLES:
    create_table_script = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        c.execute(create_table_script)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
c.close()
cnx.close()




Database yelp does not exists.
Database yelp created successfully.
Creating table restaurants: OK
Creating table reviews: OK


In [8]:
#returns true if all the necessary keys are in the restaurant dict
def all_rest_elements(restaurant):
    #this the way check if all vital keys in restaurant
    if all (k in restaurant for k in ("name","id", "rating", "price")):
        if len(restaurant['price']) > 0:
            return True
    return False


#same but for review
def all_review_elements(review):
    if all (k in review for k in ("id","restaurant_id", "rating", "time_created")):
        return True
    else:
        return False
    


In [9]:
cnx = mysql.connector.connect(**config.config)
c = cnx.cursor()

c.execute("""USE yelp""")
def populate_businesses(df):

#above should be an extra quote, jupyter broke
    for load in df:
        for restaurant in load:
            if all_rest_elements(restaurant):
                #assign ids before for formatting the sql statement
                rest_id = restaurant['id']
                name = restaurant['name']
                rating = restaurant['rating']
                price = restaurant['price']
                #sql for inserting this restaurants
                insert_str = f"""
                    INSERT INTO restaurants
                    (restaurant_id, name, rating, price) 
                    VALUES ("{rest_id}", 
                            "{name}", 
                            "{rating}", 
                            "{price}")
                    """    
                #execute and save    
                c.execute(insert_str)
                cnx.commit()
    #return dummy string
    return "Data Added"
    
populate_businesses(df)
cnx.close()

In [10]:

#use select to retrieve list of all business ids
def get_all_biz_ids():
    c, cnx = connect_to_yelp()
    select_ids = """SELECT restaurant_id
                    FROM restaurants"""
    c.execute(select_ids)
    all_ids = c.fetchall()
    return all_ids

def add_biz_id(data, biz_id):
    for review in data:
        review["restaurant_id"] = biz_id
    #data will be edited, so just return data
    return data

#retrieves all reviews and puts them into json format
def get_reviews(api_key):
    all_biz_ids = get_all_biz_ids()
    #initialize all_reviews for mapping
    all_reviews = []
    #go through each of the biz ids, get the data corresponding to it,
    #and add the biz id for foreign key use later
    for biz_id_tup in all_biz_ids:
        biz_id = biz_id_tup[0]
        #gets a dictionary from the API adding biz Id to the url
        response = yelp_review_call(api_key, biz_id)
        data = response.json()['reviews']
        #add restaurant_id
        data_with_biz = add_biz_id(data, biz_id)
        all_reviews.extend(data_with_biz)
    return all_reviews

reviews = get_reviews(config.api_key)


In [11]:
reviews[4]

{'id': 'kW1j3hwaIjgj64Hhmmj6qg',
 'url': 'https://www.yelp.com/biz/campo-azul-nolensville?adjust_creative=OGWT2CuDRacTCmIKWxKLCg&hrid=kW1j3hwaIjgj64Hhmmj6qg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=OGWT2CuDRacTCmIKWxKLCg',
 'text': "If you're absolutely desperate for Mexican food than look no further. Campo has a wide assortment of overpriced, soggy, under seasoned food; not to mention...",
 'rating': 1,
 'time_created': '2019-07-21 15:03:23',
 'user': {'id': 'ckVNVTNqg_d7e9Oqq83wDQ',
  'profile_url': 'https://www.yelp.com/user_details?userid=ckVNVTNqg_d7e9Oqq83wDQ',
  'image_url': None,
  'name': 'Cassie B.'},
 'restaurant_id': '-6osFx4NgFRHJ5NXqpHV6g'}

In [12]:
#some review text has quotes and was causing conflicts when adding to db
#replace " with \"
def replace_quotes(text):
    return text.replace(r'"', r'\"')

In [13]:
c, cnx = connect_to_yelp()

def populate_reviews(df):

    for review in reviews:
        if all_review_elements(review):
            #assign ids before for formatting the sql statement
            rev_id = review['id']
            rest_id = review['restaurant_id']
            time_stamp = review['time_created']
            rev_rating = review['rating']
            rev_text = replace_quotes(review['text'])
            #sql for inserting this restaurants
            insert_str = f"""
                INSERT INTO reviews
                (review_id, time_created, review_text, user_rating, restaurant_id) 
                VALUES ("{rev_id}", 
                        "{time_stamp}",
                        "{rev_text}",
                        "{rev_rating}", 
                        "{rest_id}")
                """    
            #execute and save    
            c.execute(insert_str)
            cnx.commit()
    #return dummy string
    return "Data Added"
    
populate_reviews(reviews)
cnx.close()


## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [14]:
c, cnx = connect_to_yelp()
def display_query(query):
    c.execute(query)
    df = pd.DataFrame(c.fetchall())
    df.columns = [column[0] for column in c.description]
    return df.head() 


def top_five():
    top_five_query = """SELECT name, rating
                    FROM restaurants
                    ORDER BY rating DESC
                    LIMIT 5"""
    return display_query(top_five_query)

top_five()

,name,rating
0,Tasty And Delicious,5.0
1,Semper Sliders,5.0
2,Little Caesars Pizza,5.0
3,Cafe Fuzion,5.0
4,Kouzina Cafe Plaza Mariachi,5.0


In [15]:
# Your code here; use a function or loop to retrieve all the results from your original request
def bottom_five():
    bottom_five_query = """SELECT name, rating
                    FROM restaurants
                    ORDER BY rating 
                    LIMIT 5"""
    return display_query(bottom_five_query)

bottom_five()

,name,rating
0,KFC,1.0
1,Pizza Hut,1.0
2,KFC,1.0
3,Pizza Hut,1.0
4,Sonic Drive-In,1.5


In [16]:
def rating_per_price():
    average_query = """SELECT price, ROUND(AVG(rating), 2) as average_rating
                    FROM restaurants
                    GROUP BY price"""
    return display_query(average_query)

rating_per_price()

,price,average_rating
0,$$$,3.98
1,$$$$,3.67
2,$$,3.59
3,$,3.49


In [17]:
def highly_rated_restaurants():
    top_query = """SELECT COUNT(*) AS top_rated
                    FROM restaurants
                    WHERE rating > 4.5"""
    return display_query(top_query)

highly_rated_restaurants()

,top_rated
0,13


In [18]:
def low_rated_restaurants():
    bottom_query = """SELECT COUNT(*) AS bottom_rated
                        FROM restaurants
                        WHERE rating < 3"""
    return display_query(bottom_query)

low_rated_restaurants()

,bottom_rated
0,127


In [19]:
def oldest_review():
    oldest_query = """SELECT review_text, time_created, user_rating
                        FROM reviews
                        ORDER BY time_created
                        LIMIT 1"""
    return display_query(oldest_query)

oldest_review()

,review_text,time_created,user_rating
0,"As far as Burger King experiences go, this one...",2011-04-06,4.0


In [20]:
def rating_for_old_review():
    old_rating_query = """SELECT name, time_created, rating
                        FROM restaurants 
                        JOIN reviews
                        ORDER BY time_created ASC
                        LIMIT 1"""
    return display_query(old_rating_query)

rating_for_old_review()

,name,time_created,rating
0,Sushiyobi,2011-04-06,3.5


In [21]:
def newest_review_for_best():
    new_best_q = """SELECT review_text, user_rating, time_created
                    FROM reviews
                    WHERE restaurant_id = (SELECT restaurant_id
                                            FROM restaurants
                                            ORDER BY rating DESC
                                            LIMIT 1)                  
                    ORDER BY time_created DESC
                    LIMIT 1"""
    return display_query(new_best_q)
newest_review_for_best()

,review_text,user_rating,time_created
0,One word: AMAZING!!!! \n\nThis had to be the b...,5.0,2019-07-01


In [22]:
def oldest_review_for_worst():
    old_worst_q = """SELECT review_text, user_rating, time_created
                    FROM reviews
                    WHERE restaurant_id = (SELECT restaurant_id
                                            FROM restaurants
                                            ORDER BY rating
                                            LIMIT 1)                  
                    ORDER BY time_created
                    LIMIT 1"""
    return display_query(old_worst_q)
oldest_review_for_worst()

,review_text,user_rating,time_created
0,Went there at 9:30 two days in a row and the d...,1.0,2018-08-27


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [23]:
def graph_price_count():
    average_query = """SELECT price, COUNT(*) AS total
                    FROM restaurants
                    GROUP BY price"""
    return display_query(average_query)

graph_price_count()

,price,total
0,$$$,32
1,$$$$,6
2,$$,512
3,$,345


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [24]:
#Your code here


## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!